In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.callbacks import Callback
import time

# Custom callback to print training and testing accuracy
class AccuracyCallback(Callback):
    def _init_(self, x_train, y_train, x_test, y_test):
        super()._init_()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.x_train, self.y_train, verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        print(f"Epoch {epoch + 1}/{self.params['epochs']}, "
              f"Training Accuracy: {train_accuracy:.4f}, "
              f"Testing Accuracy: {test_accuracy:.4f}")

# Load your CSV file
file_path = r"C:\Users\02gsh\Desktop\datasets\all_read.csv"
df = pd.read_csv(file_path)

# Assuming your CSV file has columns/features and a target column named 'labels'
X = df.drop('labels', axis=1)
y = df['labels']

# Preprocess the data (standardization and label encoding)
scaler = StandardScaler()
X = scaler.fit_transform(X)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
# classifier instead of model
classifier = Sequential([
    Dense(128, input_shape=(x_train.shape[1],), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
classifier.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



# Train the model
history = classifier.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

# Evaluate the model on training data to get final training accuracy
final_train_loss, final_train_accuracy = classifier.evaluate(x_train, y_train)
print("Final Training Accuracy:", final_train_accuracy)

# Evaluate the model on testing data
test_loss, test_accuracy = classifier.evaluate(x_test, y_test)
print("Final Testing Accuracy:", test_accuracy)

48/48 [==============================] - 0s 3ms/step - loss: 0.3380 - accuracy: 0.9567
Final Training Accuracy: 0.9566929340362549
12/12 [==============================] - 0s 3ms/step - loss: 0.4829 - accuracy: 0.9081
Final Testing Accuracy: 0.9081364870071411


In [ ]:
#CSA
def fitness(variables_values=[]):
    classifier = Sequential([
        Dense(128, input_shape=(x_train.shape[1],), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Assuming you have already defined x_train, y_train, x_test, y_test, and label_encoder
    classifier.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    test_loss, accuracy = classifier.evaluate(x_test, y_test)
    return accuracy

def initial_population(population_size = 30, min_values = [5,1], max_values = [120,3], target_function = fitness):
    population = np.zeros((population_size, len(min_values) + 1))
    for i in range(0, population_size):
        for j in range(0, len(min_values)):
            population[i,j] = random.uniform(min_values[j], max_values[j])
        population[i,-1] = target_function(population[i, 0:population.shape[1]-1])
    return population
def update_position(population, ap, fL, min_values = [5,1], max_values = [120,3], target_function = fitness):
    for i in range(0, population.shape[0]):
        idx  = [i for i in range(0, population.shape[0])]
        idx.remove(i)
        idx  = random.choice(idx)
        rand = int.from_bytes(os.urandom(8), byteorder = 'big') / ((1 << 64) - 1)
        for j in range(0, len(min_values)):
            if (rand >= ap):
                rand_i           = int.from_bytes(os.urandom(8), byteorder = 'big') / ((1 << 64) - 1)
                population[i, j] = np.clip(population[i, j] + rand_i*fL*( population[idx, j] - population[i, j]), min_values[j], max_values[j])
            else:
                population[i,j] = random.uniform(min_values[j], max_values[j])
        population[i,-1] = target_function(population[i, 0:population.shape[1]-1])
    return population

def crow_search_algorithm(population_size=30, ap=0.02, fL=2, min_values=[5, 1], max_values=[120, 3], iterations=30, target_function=fitness, stagnation_limit=10):
    rem = []
    count = 0
    stagnation_count = 0
    population = initial_population(population_size, min_values, max_values, target_function)
    best_ind = np.copy(population[population[:, -1].argsort()][0, :])
    best_fitness = best_ind[-1]

    while count <= iterations and stagnation_count < stagnation_limit:
        population = update_position(population, ap, fL, min_values, max_values, target_function)
        for i in range(0, population_size):
            value = np.copy(population[i, :])
            rem.append(value)
            if best_ind[-1] < value[-1]:
                best_ind = value
                best_fitness = value[-1]
                stagnation_count = 0
            else:
                stagnation_count += 1
        count += 1

    best_ind[1] = int(ceil(best_ind[1]))
    return best_ind, rem

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.utils import resample
from math import *
import math
import random
import os

start_time=time.time()


resultat_DNN=crow_search_algorithm()
resultat_DNN[0]#best position

training_time = end_time - start_time
print("Training time:", training_time, "seconds")

12/12 [==============================] - 0s 3ms/step - loss: 0.4951 - accuracy: 0.8871


In [1]:
accuracy_DNN_withCSA=resultat_DNN[0][2]
accuracy_DNN_withCSA



NameError: name 'resultat_DNN' is not defined